In [1]:
# imports
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
drive.mount('/content/drive')
# file_path = 'reg_pbp_2019.csv'
file_path = "/content/drive/MyDrive/MSAI349/349_project/reg_pbp_2019.csv"
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-ff5b16cd38ce>:4: DtypeWarning: Columns (167,168,169,170,175,176,179,180,183,184,189,190,191,192,195,196,204,205,206,219,220,221,222,223,224,227,228,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


## Preprocessing

In [5]:
df.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,35,2019090500,CHI,GB,GB,away,CHI,CHI,35.0,2019-09-05,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,50,2019090500,CHI,GB,GB,away,CHI,GB,75.0,2019-09-05,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,71,2019090500,CHI,GB,GB,away,CHI,GB,75.0,2019-09-05,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,95,2019090500,CHI,GB,GB,away,CHI,GB,75.0,2019-09-05,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,125,2019090500,CHI,GB,GB,away,CHI,GB,85.0,2019-09-05,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


In [6]:
list(df.columns)

['play_id',
 'game_id',
 'home_team',
 'away_team',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential',
 'posteam_score_post',
 'defteam_score_post',
 'score_differential_post',
 'no_sc

In [7]:
# take only columns that are relevant to our problem
df = df[[# 'play_id', not going to know this information
 'game_id', # very large scale and handled by index anyway
 'home_team',
 'away_team',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 # 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'drive',
 'qtr',
 'down',
 'goal_to_go',
 # 'time', categorical time, time is in game_seconds_remaining
 # 'yrdln', categorical for side_of_field
 'ydstogo',
 'ydsnet',
 'play_type',
 'no_huddle',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential'
 ]]

In [8]:
# take only plays that are pass or run plays
pass_run_df = df[(df['play_type'] == 'run') | (df['play_type'] == 'pass')]

In [9]:
# save preprocessed df into a csv
new_file_path = 'pass_run_pbp_2019.csv'
pass_run_df.to_csv(new_file_path, index=False)

In [10]:
pass_run_df_cleaned = pass_run_df.dropna()

labels = pass_run_df_cleaned["play_type"]
map = {"pass": 1, "run": 0}
labels = labels.replace(map)

new_file_path = '/content/drive/MyDrive/labels.csv'
# labels.to_csv(new_file_path, index=False)
labels.isna().sum()

0

In [11]:
features = pass_run_df_cleaned.drop("play_type", axis=1)

# maps for categorical features
team_map = {"CHI": 0, "CAR": 1, "PHI": 2, "NYJ": 3, "MIN": 4, "MIA": 5, "JAX": 6, "CLE": 7, "LAC": 8, "SEA": 9, "TB": 10,
            "DAL": 11, "ARI": 12, "NE": 13, "NO": 14, "OAK": 15, "BAL": 16, "WAS": 17, "TEN": 18, "PIT": 19, "NYG": 20,
            "CIN": 21, "DET": 22, "GB": 23, "HOU": 24, "LA": 25, "DEN": 26, "ATL": 27, "BUF": 28, "KC": 29, "IND": 30,
            "SF": 31}
team_type_map = {"home": 0, "away": 1}
side_map = {'MID': 50}
half_map = {'Half1': 0, 'Half2': 1, 'Overtime': 2}

features = features.replace(team_map)
features = features.replace(team_type_map)
features = features.replace(side_map)
features = features.replace(half_map)

new_file_path = '/content/drive/MyDrive/features.csv'
# features.to_csv(new_file_path, index=False)

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

names = features.columns
scaled_features = pd.DataFrame(scaled_features, columns=names)


new_file_path = '/content/drive/MyDrive/scaled_features.csv'
# scaled_features.to_csv(new_file_path, index=False)

In [12]:
scaled_features.isna().sum()

game_id                       0
home_team                     0
away_team                     0
posteam                       0
posteam_type                  0
defteam                       0
side_of_field                 0
yardline_100                  0
quarter_seconds_remaining     0
half_seconds_remaining        0
game_seconds_remaining        0
game_half                     0
drive                         0
qtr                           0
down                          0
goal_to_go                    0
ydstogo                       0
ydsnet                        0
no_huddle                     0
home_timeouts_remaining       0
away_timeouts_remaining       0
posteam_timeouts_remaining    0
defteam_timeouts_remaining    0
total_home_score              0
total_away_score              0
posteam_score                 0
defteam_score                 0
score_differential            0
dtype: int64

In [13]:
labels = labels.reset_index()
scaled_features = scaled_features.reset_index()

In [14]:
lstm_data = pd.concat([scaled_features, labels], axis=1)

lstm_data = lstm_data.drop('index', axis=1)
lstm_data

,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,quarter_seconds_remaining,half_seconds_remaining,...,home_timeouts_remaining,away_timeouts_remaining,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,play_type
0,0.0,0.000000,0.741935,0.741935,1.0,0.000000,0.46,0.755102,1.000000,1.000000,...,1.00,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.50000,0
1,0.0,0.000000,0.741935,0.741935,1.0,0.000000,0.46,0.755102,0.970000,0.985000,...,1.00,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.50000,1
2,0.0,0.000000,0.741935,0.741935,1.0,0.000000,0.46,0.755102,0.916667,0.958333,...,1.00,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.50000,1
3,0.0,0.000000,0.741935,0.000000,0.0,0.741935,0.00,0.520408,0.836667,0.918333,...,1.00,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.50000,0
4,0.0,0.000000,0.741935,0.000000,0.0,0.741935,0.46,0.469388,0.797778,0.898889,...,1.00,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.50000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31970,1.0,0.290323,1.000000,0.290323,0.0,1.000000,0.62,0.112245,0.046667,0.023333,...,0.25,0.5,0.000000,0.333333,0.396226,0.440678,0.355932,0.440678,0.44898,1
31971,1.0,0.290323,1.000000,0.290323,0.0,1.000000,0.62,0.040816,0.024444,0.012222,...,0.25,0.5,0.000000,0.333333,0.396226,0.440678,0.355932,0.440678,0.44898,1
31972,1.0,0.290323,1.000000,0.290323,0.0,1.000000,0.62,0.040816,0.016667,0.008333,...,0.25,0.5,0.000000,0.333333,0.396226,0.440678,0.355932,0.440678,0.44898,1
31973,1.0,0.290323,1.000000,0.290323,0.0,1.000000,0.62,0.040816,0.013333,0.006667,...,0.25,0.5,0.000000,0.333333,0.396226,0.440678,0.355932,0.440678,0.44898,1


In [15]:
new_file_path = '/content/drive/MyDrive/MSAI349/349_project/lstm_data.csv'
lstm_data.to_csv(new_file_path, index=False)